In [19]:
import numpy as np
import pandas as pd

## Conjuntos 
Del modelo de optimización

| **Término**  | **Descripción**                |
|--------------|--------------------------------|
| $cacopios$   | Conjunto de centros de acopio  |
| $clientes$   | Conjunto de clientes           |
| $productos$  | Conjunto de productos          |

In [20]:
cacopios = 'CAcopios'
clientes = 'Clientes'
productos = 'Productos'

In [21]:
csv_base = './csv/'

routes = {
    # Capacidades de centros de acopio
    'cic': 'CapInvCA.csv',
    'cictest': 'CapInvCAtest.csv',
    # Costo de inventario del centro de acopio
    'cia' : 'CostoInvCA.csv',
    'ciatest' : 'CostoInvCAtest.csv',
    # Costo de transporte del centro de acopio al cliente
    'ctac' : 'CostoTransCAClie.csv',
    'ctactest' : 'CostoTransCAClietest.csv'
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

## Centros de acopio

In [22]:
cacopios_df = pd.read_csv(csv_base + cacopios + '.csv')
# cacopios_df = pd.read_csv(csv_base + cacopios + 'test' + '.csv')
cacopios_df

,CAcopios
0,San Juan del Cesar
1,Riohacha
2,Maicao
3,Guamal
4,Pivijay
5,Nueva Granada
6,Monteria
7,Chinu
8,Planeta rica


## Clientes

In [23]:
clientes_df = pd.read_csv(csv_base + clientes + '.csv')
# clientes_df = pd.read_csv(csv_base + clientes + 'test' + '.csv')
clientes_df

,Clientes
0,Distribuidor1
1,Distribuidor2
2,Distribuidor3
3,Tienda1
4,Tienda2
5,Mercado1
6,Mercado2
7,Mercado3
8,Panaderia1
9,Panaderia2


## Productos

In [24]:
productos_df = pd.read_csv(csv_base + productos + '.csv')
productos_df.head()

,Productos
0,Queso duro
1,Queso blando


# Escenarios de prueba
Tomando demandas por cada centro de acopio, se definen distintos escenarios de posibles casos.

**Escenario 1**: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

**Escenario 2**: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

**Escenario 3**: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Demandas
Generando demandas para cada cliente, se usarán como entrada para el modelo

In [25]:
demanda_clientes = clientes_df[clientes].drop_duplicates()

demandas = []

for client in demanda_clientes:
    demandas.append(20 * np.random.randint(10))

demanda_clientes = pd.DataFrame(demanda_clientes)

demanda_clientes['Demanda'] = pd.DataFrame(demandas)
demanda_clientes

,Clientes,Demanda
0,Distribuidor1,100
1,Distribuidor2,100
2,Distribuidor3,0
3,Tienda1,80
4,Tienda2,80
5,Mercado1,20
6,Mercado2,0
7,Mercado3,60
8,Panaderia1,160
9,Panaderia2,20


### Capacidades 
Obteniendo las capacidades de cada centro de acopio

In [26]:
cap_df = pd.read_csv(csv_base + routes['cic'], **csv_config)
# cap_df = pd.read_csv(csv_base + routes['cictest'], **csv_config)

cap_df

,CAcopios,Productos,CapInvCA
0,San Juan del Cesar,Queso duro,430
1,San Juan del Cesar,Queso blando,320
2,Riohacha,Queso duro,60
3,Riohacha,Queso blando,90
4,Maicao,Queso duro,190
5,Maicao,Queso blando,0
6,Guamal,Queso duro,140
7,Guamal,Queso blando,170
8,Pivijay,Queso duro,200
9,Pivijay,Queso blando,280


#### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [27]:
cia_df = pd.read_csv(csv_base + routes['cia'], **csv_config)
# cia_df = pd.read_csv(csv_base + routes['ciatest'], **csv_config)
cia_df.head()

,CAcopios,Productos,CostoInvCA
0,San Juan del Cesar,Queso duro,1000
1,San Juan del Cesar,Queso blando,1500
2,Riohacha,Queso duro,500
3,Riohacha,Queso blando,800
4,Maicao,Queso duro,900


#### Costos de Transporte
$$
  CTransporte_t = \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} CostoTransAcopioClie_{pjkt} \qquad \forall_t \in T
$$

In [28]:
ctac_df = pd.read_csv(csv_base + routes['ctac'], **csv_config)
# ctac_df = pd.read_csv(csv_base + routes['ctactest'], **csv_config)
ctac_df.head()

,CAcopios,Clientes,CostoTransCAClie
0,San Juan del Cesar,Distribuidor1,68000
1,San Juan del Cesar,Distribuidor2,38000
2,San Juan del Cesar,Distribuidor3,56000
3,San Juan del Cesar,Tienda1,32000
4,San Juan del Cesar,Tienda2,95000


## Esquema de representación
Se recibe la demanda $DemandaClie_{pkt}$ como parámetro de entrada.

Se representa la cantidad $n$ de centros de acopio como un vector, donde se asigna una cantidad $x_i$ para cada centro de acopio $J_i$, la cantidad asignada representa la demanda que va a ser suplida por ese centro de acopio.
$$
\begin{array} {|r|r|r|r|r|r|}
    \hline x_0 & x_1 & x_2 & x_3 & \cdots & x_n \\
    \hline
\end{array}
\quad \therefore \quad x_i = AC_{pjkt}
$$
Donde se aplican las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{n} x_i &= DemandaClie_{pkt} \\
    x_i &\leq InvCA_{pjt}
\end{align*}
$$

In [29]:
X = np.zeros(cacopios_df.shape)
# Se toma la primera demanda como parámetro de entrada
demanda = demanda_clientes.loc[4, 'Demanda']

cap_queso = cap_df[cap_df[productos] == 'Queso duro']

for i in range(X.shape[0]):
    rand = np.random.randint(demanda)
    cap = cap_df.loc[i, 'CapInvCA']
    print(cap)
        
    if i == X.size - 1 and demanda != 0:
        X[i] = demanda
        demanda -= demanda
    else:
        if rand > cap:
            X[i] = cap
            demanda -= cap
        else:
            X[i] = rand
            demanda -= rand
    
suma_demanda = X.sum()
    
# print(X)
# print(f'Suma de la distribución: {suma_demanda}')

430
320
60
90
190
0
140
170
200


### Cálculo de costos en función objetivo

## Función objetivo
$$
Min(F) = CInventario + CTransporte
$$

In [30]:
# F(x)
def f(x):
    delta = 0
    # print(f'{x} -> Suma: {x.sum()}')

    for idx in range(x.shape[0]):
        ca = cacopios_df.loc[idx]

        if x[idx] == 0.:
            continue

        # Costos de Inventario
        costo_inv_ca = cia_df[(cia_df[cacopios] == ca[cacopios]) & (cia_df[productos] == 'Queso duro')]
        # Costos de Transporte
        costo_trans_ca = ctac_df[(ctac_df[cacopios] == ca[cacopios]) & (ctac_df[clientes] == 'Distribuidor1')]

        delta += x[idx] * costo_inv_ca['CostoInvCA'].values[0]
        delta += np.array(costo_trans_ca['CostoTransCAClie'].values[0])
    
    # if np.round(x.sum()) != suma_demanda:
    #     delta *= 2
    
    return delta

print(f'Costo: {f(X)}')

Costo: [313700.]


Definición del vector con la primera posible solución y los límites de capacidad para cada centro de acopio

In [31]:
bounds = [(0, x_i) for x_i in cap_queso['CapInvCA']]
x0 = [x[0] for x in X]

## scipy

**LinearConstraint**: $ lb \leq A*v[x] \leq ub $

**NonLinearConstraint**: $ lb \leq g(x) \leq ub $

In [32]:
# from scipy.optimize import LinearConstraint
# from scipy.optimize import NonlinearConstraint
# 
# cons = {'constraints': LinearConstraint(np.ones(X.shape[0]), suma_demanda, suma_demanda)}
# 
# g = lambda x : x.sum()
# non_cons = {'constraints': NonlinearConstraint(g, suma_demanda, suma_demanda)}


In [33]:
# from scipy.optimize import dual_annealing
# from scipy.optimize import differential_evolution

# result = dual_annealing(
#     f,
#     bounds=bounds,
#     minimizer_kwargs=cons,
#     # maxiter=200,
#     x0=np.array(x0)
# )
# print(result)

## pymoo

In [34]:
from pymoo.core.problem import ElementwiseProblem

xu = np.asarray([b for _, b in bounds])

class Queso(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=len(x0),
            n_obj=1,
            n_eq_constr=1,
            # n_ieq_constr=1,
            xl=0,
            xu=xu,
            vtype=int
        )
        
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = f(X)
        out['H'] = suma_demanda - (np.sum(x))

problem = Queso()

**Genetic Algorithm**: $ (\mu + \lambda) \quad \therefore \quad \mu = 100 $

In [39]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

algorithm = GA(
    pop_size=100,
    eliminate_duplicates=True
)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f'Optimal value: {res.F} with:\n\t X = {res.X}\n\t ')

Optimal value: [313700.] with:
	 X = [ 0.97225128  1.79209671  2.41032107  4.73617132  8.01374862  5.83548281
 42.52320537  3.16077222 10.5559832 ]
	 


**Differential evolution**: $ v = x_{\pi_1} + F \cdot (x_{\pi_2} - x_{\pi_3} ) $

In [38]:
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize

algorithm = DE(
    pop_size=10,
    sampling=LHS(),
    variant='DE/rand/1/bin',
    CR=0.3,
    dither='vector',
    jitter=False
)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f'Optimal value: {res.F} with:\n\t X = {res.X}\n\t ')

Optimal value: None with:
	 X = None
	 
